## Introdução

Nesse colab vou explicar como treinar um modelo de detecção de objetos YOLOv11 em um PC local, um modelo yolo pode ser treinado em um colab como este também, mas para o desafio explicarei como treinar o yolo localmente.

Vou falar sobre o processo de preparação de dados, instalação do Ultralytics(https://docs.ultralytics.com/pt/), treinamento de um modelo e execução de inferencia com um script Python personalizado para o uso da transmissão do hardware ESP32Cam.

Estarei mostrando o passo a passo para windows, mas com poucas modificações pode ser usado em sistemas Linux.

OBSERVAÇÃO : Uma placa de vídeo NVIDIA compatível com CUDA é altamente recomendada para modelos de treinamento.

### Etapa 1 - Instalar o Anaconda

Primeiro, precisamos instalar o Anaconda( https://anaconda.com/download ), uma ótima ferramenta para criar e gerenciar ambientes Python. Ele permite que você instale bibliotecas Python sem se preocupar com conflitos de versão com instalações existentes no seu sistema operacional.

### Etapa 2 - Crie um novo ambiente e instale o Ultralytics

Agora, vamos criar um novo ambiente Anaconda Python para a biblioteca Ultralytics. Siga as seguintes etapas: Crie um novo ambiente executando:

In [ ]:
conda create --name yolo11-env python=3.12 -y

In [ ]:
conda activate yolo11-env

In [ ]:
pip install ultralytics

Ao instalar o Ultralytics, o gerenciador de pacotes Python também instala diversas outras bibliotecas poderosas, como OpenCV, Numpy e PyTorch.

Para executar o treinamento na GPU, precisamos usar a versão do PyTorch habilitada para GPU. Execute o seguinte comando para instalar a versão do PyTorch para GPU:

OBSERVAÇÃO: O PyTorch lança regularmente versões atualizadas com suporte para versões mais recentes do CUDA. O comando acima instala o PyTorch com o CUDA v12.4. Versão mais recente em ( https://pytorch.org/get-started/locally/ )

In [ ]:
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

### Etapa 3 - Reúna e rotule as imagens

O primeiro passo para treinar qualquer modelo de aprendizado de máquina é construir um conjunto de dados. Para modelos de detecção de objetos, isso significa coletar e rotular centenas a milhares de imagens dos objetos que você deseja detectar.

onjuntos de dados podem ser coletados online a partir de dados preexistentes em sites como Kaggle(https://www.kaggle.com/) , Open Images V7(https://storage.googleapis.com/openimages/web/index.html) ou Roboflow Universe (https://universe.roboflow.com/).

Normalmente eu usaria o roboflow para construção e categorização dos conjuntos de dados para o treinamento, mas como tratamos de um desafio industrial, acredito que o roblow por mais que seja facil, não garante segurança e controle de dados (além de conjutos de dados capturados manualmente tendem a ter uma melhor performace no desempenho). pontos que são muito importantes para empresas em geral. por isso optei por usar o Label Studio (https://github.com/HumanSignal/labelImg)

Existem duas **etapas principais** para desenvolver um conjunto de dados de treinamento personalizado eficaz para modelos de detecção de objetos:

1. Coletando imagens
2. Rotulagem de objetos

In [ ]:
pip install -U label-studio

In [ ]:
label-studio

Apos fazer a instalação do aplicativo do label studio e executa-lo ele será aberto para carregar e classificar as imagens.

Então export os dados no formato YOLO, e tera um arquivo .zip

- Uma pasta “imagens” contendo as imagens
- Uma pasta “labels” contendo os rótulos no formato de anotação YOLO
- Um arquivo labelmap “classes.txt” que contém a lista de classes

### Etapa 4 - Configurar estrutura de pastas

O Ultralytics requer uma estrutura de pasta específica para armazenar dados de treinamento para modelos. A pasta raiz é chamada de "dados". Dentro dela, há duas pastas principais:


- train : Contém as imagens e rótulos que são fornecidos ao modelo durante o processo de treinamento.
- Validação : Contém imagens e rótulos que são usados ​​periodicamente para testar o modelo durante o treinamento. Ao final de cada período de treinamento, o modelo é executado nessas imagens para determinar métricas como precisão, recall e mAP.

80% das imagens em um conjunto de dados são divididas na pasta "train", enquanto 20% são divididas na pasta "validation"

### Etapa 5 - Configurar treinamento

Precisamos criar o arquivo data.yaml de configuração de treinamento do Ultralytics.

In [1]:
# algo do tipo

path: C:\Users\<username>\Documents\yolo\data
train: train\images
val: validation\images

nc: 2

names: ["Milho", "Erva Daninha"]

### Etapa 6 - Treine o modelo!

Há uma variedade de tamanhos de modelos disponíveis para treinamento. (https://docs.ultralytics.com/pt/models/yolo11/)

Os **parâmetros importantes a serem considerados** são mAP, parâmetros e FLOPs. O valor de mAP indica a precisão relativa do modelo (ou seja, sua eficiência na detecção de objetos) em comparação com os outros modelos. As colunas parâmetros e FLOPs indicam o tamanho do modelo: quanto maiores esses valores, mais recursos computacionais o modelo exigirá.

Os modelos YOLO são normalmente treinados e inferidos com **uma resolução de 640x640**. A resolução de entrada do modelo pode ser ajustada usando o argumento "–imgsz" durante o treinamento. A resolução do modelo tem um grande impacto na velocidade e precisão do modelo: um modelo com resolução mais baixa terá maior velocidade, mas menor precisão.

Em aprendizado de máquina, **uma "época"** é uma única passagem pelo conjunto de dados de treinamento completo. Em cada época, todas as imagens do conjunto de dados são inseridas no modelo, e o algoritmo de aprendizado atualiza os pesos internos do modelo para melhor se adequarem aos dados na imagem. A melhor quantidade de épocas a ser usada depende do tamanho do conjunto de dados.



**Treinando o modelo**

O comando “yolo detect train” é usado para executar o treinamento. Ele possui alguns argumentos importantes:

- **dados** : especifica o caminho para o arquivo de configuração de treinamento (que configuramos na Etapa 5)
- **modelo** : especifica qual arquitetura de modelo treinar (ex.: "yolo11s.pt", "yolo11l.pt"). Você também pode treinar os modelos YOLOv5 e YOLOv8 substituindo "yolo11" por "yolov5" ou "yolov8" (ex.: "yolov5l.pt" ou "yolov8s.pt").
- **épocas** : define o número de épocas para treinar
- **imgsz** : define a dimensão de entrada (ou seja, resolução) do modelo YOLO

In [ ]:
yolo detect train data=data.yaml model=yolo11s.pt epochs=60 imgsz=640

O algoritmo de treinamento analisará as imagens nos diretórios de treinamento e validação e, em seguida, iniciará o treinamento do modelo.

O treinamento terminará ao atingir o número de períodos especificado por "epochs".

### Etapa 7 - Executar modelo

Depois de terminar o treinamento do modelo, vamos usar o treinamento para capturar as imagens do endpoint da camera do hardware do ESP32Cam.


Aqui vou deixar um codigo de exemplo da aplicação para uso.

In [ ]:
import cv2
import numpy as np
import requests
from ultralytics import YOLO
import time

# Carregar modelo YOLO
model_path = "runs/detect/train/weights/best.pt"  # Substitua pelo seu caminho real
model = YOLO(model_path)
labels = model.names

# Definir cores para as bounding boxes
bbox_colors = [(164,120,87), (68,148,228), (93,97,209), (178,182,133), (88,159,106),
               (96,202,231), (159,124,168), (169,162,241), (98,118,150), (172,176,184)]

# Loop de inferência
while True:
    try:
        # Pegar frame do endpoint /stream
        response = requests.get("http://127.0.0.1:5002/stream", stream=True)
        if response.status_code != 200:
            print("Erro ao acessar o stream.")
            break

        img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        frame = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        if frame is None:
            print("Frame inválido.")
            continue

        # Início da contagem de tempo
        t_start = time.perf_counter()

        # Rodar inferência
        results = model(frame, verbose=False)
        detections = results[0].boxes

        object_count = 0

        for i in range(len(detections)):
            xyxy = detections[i].xyxy.cpu().numpy().squeeze().astype(int)
            xmin, ymin, xmax, ymax = xyxy
            classidx = int(detections[i].cls.item())
            classname = labels[classidx]
            conf = detections[i].conf.item()

            if conf > 0.5:
                color = bbox_colors[classidx % 10]
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
                label = f'{classname}: {int(conf*100)}%'
                cv2.putText(frame, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                object_count += 1

        # Calcular FPS
        t_stop = time.perf_counter()
        fps = 1 / (t_stop - t_start)
        cv2.putText(frame, f'FPS: {fps:.2f}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f'Objetos: {object_count}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # Mostrar frame
        cv2.imshow("YOLO Stream Detection", frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    except Exception as e:
        print(f"Erro durante a execução: {e}")
        break

cv2.destroyAllWindows()
